In [ ]:
import pandas as pd
import geopandas as gpd
import mytools
import math,simplekml

# ----------------------------------空洞制作------------------------------------

# 导入数据和导出目录

In [ ]:
out_file = 'e:/'

In [ ]:
use = pd.read_clipboard()

In [ ]:
data = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.data')
# data = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/覆盖空添加场景区分大中小微.data')

In [ ]:
use.shape

In [ ]:
use.head(1)

In [ ]:
# use = pd.read_clipboard()

In [ ]:
data.to_clipboard()

In [ ]:
data.head(1)

In [ ]:
data_use = data.merge(use,left_on='id',right_on='空洞标识')

In [ ]:
data_use.shape

In [ ]:
data_use.head(1)

In [ ]:
data_use[['lon','lat']] = data_use.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#

In [ ]:
data_use.columns

In [ ]:
use_columns=['city', 'type', 'geometry', '面积', 'lon', 'lat', '区域类型', 'id', '制式',
       'distance', '标准面积', '空洞面积比例', '空洞级别', '空洞标识', '区县-确认']

In [ ]:
# use_columns =['地市', '区县', 'geometry', 'type', '面积', 'lon', 'lat', '区域类型', 'id', '制式',
#        'distance', '标准面积', '空洞面积比例', '空洞级别']

In [ ]:
# data_use = data.copy()

# data_use.shape

In [ ]:
# data_use.to_pickle('d:/aohaiying.data')

In [ ]:
# data_use = pd.read_pickle('d:/aohaiying.data')

In [ ]:
# data_use.drop(columns='geometry').to_clipboard()

# 设置变量

In [ ]:
#- 变量 - 设置经纬度和参与循环的字段
lon,lat,use_id,city = ['lon', 'lat','id','区县-确认']

In [ ]:
#- 颜色 - 按照字符设置颜色
colour_data = '空洞级别'
colour_dic = {'大':'red','中':'yellow','小':'blue','微':'green','other':'white'}

In [ ]:
#- 标注 -添加标注列
de_col = use_columns
de_col.remove('geometry')

In [ ]:
#- 视野 - 在1400*900的分辨率下  view = 70米对应：125公里（差不多能看到武汉主城区）  700对应1200公里（差不多能看到湖北省）
view = 50000

# 整理数据

In [ ]:
data_use.head(1)

In [ ]:
#添加标注列
data_use['description']=''
for inde_1, name_1 in enumerate(de_col):
    data_use['linshi']=de_col[inde_1]+' : '+data_use[de_col[inde_1]].astype('str')+'\n'
    data_use['description'] = data_use['description']+data_use['linshi']
    data_use.drop(columns='linshi')

In [ ]:
data_sectors= data_use.copy()
#添加lod的polygon
data_sectors['lod_dis']= view
data_sectors['lon1'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(45* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat1'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(45* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['lon2'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(225* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat2'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(225* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['list_data']=[([lon_1, lat_1,lon_2, lat_2]) 
 for lon_1, lat_1,lon_2, lat_2 in zip(data_sectors['lon1'],data_sectors['lat1'],data_sectors['lon2'],data_sectors['lat2'])]

In [ ]:
# df = data_sectors.copy()
# #添加颜色列-数字
# df = data_sectors.copy()
# df['colour']=1
# df['colour'] = pd.cut(df['colour'],bins=[-0.1, 0.7, 0.936, 1.1],labels =['red', 'yellow', 'green'])
# df['colour'] = df['colour'].astype('object')
# # df.loc[df[rsrp_count]<100,'colour']='white'

In [ ]:
df = data_sectors.copy()
#添加颜色列-文字
df['colour']='white'
for name_col in colour_dic:
    df.loc[df[colour_data]==name_col,'colour'] =colour_dic[name_col]

In [ ]:
#最终使用的列
df_use = df.reindex(columns=[city,use_id,colour_data, 'list_data','colour','description','geometry'])   

# 生成图层

In [ ]:
def make_kml(data_t,name='红色栅格',cc='ff0000ff',xiankuan=0,namea='grid_id',list_data = 'list_data',description='description',minlodpixels=1):
    print(data_t.shape,cc,xiankuan,namea,list_data,description,minlodpixels)
    style = simplekml.Style()
    style.linestyle.color = simplekml.Color.changealphaint(250, cc)  # 最终线条上色
    style.polystyle.outline = xiankuan
    style.polystyle.color = simplekml.Color.changealphaint(250, cc )  # 最终形状上色
    lod1 = simplekml.Lod(minlodpixels=minlodpixels, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
    grid_red = kml.newfolder(name=name)
    for grid,list_data ,description_str,geo in zip(data_t[namea],data_t[list_data],data_t[description],data_t['geometry']):
#         pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])#栅格使用
        pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list(geo.exterior.coords),innerboundaryis=[po.coords for po in list(geo.interiors)])
        pol_r.description = description_str
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        lon_dd,lat_dd,lon1_dd,lat1_dd = list_data
        latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                              minaltitude=None, maxaltitude=None, altitudemode=None)
        pol_r.region.latlonaltbox = latlonaltbox
        pol_r.region.lod = lod1
        pol_r.style=style

In [ ]:
#如果是多个文件就使用这个，一个文件不用
# f = mytools.othern.file_name_paths('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/修正覆盖半径/各个地市覆盖图层/')

In [ ]:
# for name_f in f:#如果是多个文件就使用这个，一个文件不用
#     grid_t = pd.read_pickle(name_f)
#     grid_groupby = grid_t.groupby(['所属地区'])
for cquxian,data_quxian in df_use.groupby(city):
    for name_region,data_region in data_quxian.groupby(colour_data):
        kml = simplekml.Kml()
    #         if not os.path.exists('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/修正覆盖半径/各个地市覆盖图层/{}'.format(name_region)):
    #             os.makedirs('G:/1-规划/评估/覆盖空洞/2020年6月/2-程序输出/修正覆盖半径/各个地市覆盖图层/{}'.format(name_region))
        file = '{}{}空洞_{}_{}.kmz'.format(out_file,cquxian,name_region,data_region.shape[0])
        print('{}开始'.format(file))
        for name_c,data_t in data_region.groupby('colour'):#red:ff0000ff,green:ff008000,yellow:ff00ffff,white:ffffffff,bule:ffff0000
            if 'red' ==name_c:
                make_kml(data_t,name='红色{}个'.format(data_t.shape[0]),cc='ff0000ff',namea=use_id,minlodpixels=50,xiankuan=1)
            elif 'blue' == name_c:
                make_kml(data_t,name='蓝色{}个'.format(data_t.shape[0]),cc='ffff0000',namea=use_id,minlodpixels=50,xiankuan=1)
            elif 'yellow' == name_c:
                make_kml(data_t,name='黄色{}个'.format(data_t.shape[0]),cc='ff00ffff',namea=use_id,minlodpixels=50,xiankuan=1)
            elif 'white' == name_c:
                make_kml(data_t,name='白色{}个'.format(data_t.shape[0]),cc='ffffffff',namea=use_id,minlodpixels=50,xiankuan=1)
            elif 'green'  == name_c:
                make_kml(data_t,name='绿色{}个'.format(data_t.shape[0]),cc='ff008000',namea=use_id,minlodpixels=50,xiankuan=1)
            kml.save(file) if 'kml' in file else kml.savekmz(file, False)
        print('生成图层成功存放在{}'.format(file))

In [ ]:
mytools.sound()
print('自爆')
assert 0 

# ----------------------------------修正后的扇区------------------------------------

# 导入数据和导出目录

In [ ]:
out_file = 'G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/'

In [ ]:
data = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/xiaoqufugaimoxing.data')

In [ ]:
data = data.reset_index()

In [ ]:
data['type'] = data.type

In [ ]:
#多边形的质心
data[['lon','lat']] = data.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#
#or
# data.rename(columns={'RRU经度':'lon','RRU纬度':'lat'})

In [ ]:
data = data.loc[data['归属区县']=='东湖高新开发区']

In [ ]:
data = data.rename(columns={'所属地区':'city'})

In [ ]:
data_use = data[[ '小区CGI(*)', '小区中文名', 'city', 'RRU经度', 'RRU纬度', '天线挂高', '方位角',
       '工作频段', '归属区县', '厂家名称', '区域类型', 'distance', '覆盖模型距离', '30米覆盖模型距离',
       '使用覆盖半径', 'distance_use', 'distance_yb', 'h_use', '高度建筑比',
       'distance_use_3', 'geometry', 'type', 'lon', 'lat']]

# 设置变量

In [ ]:
#- 变量 - 设置经纬度和参与循环的字段
lon,lat,use_id,city = ['lon','lat','city','city']

In [ ]:
# #- 颜色 - 按照字符设置颜色
# colour_data = 'city'
# colour_dic = {'大':'red','中':'yellow','小':'blue','微':'green','other':'white'}

In [ ]:
#- 标注 -添加标注列
de_col = [ '小区CGI(*)', '小区中文名', 'city', 'RRU经度', 'RRU纬度', '天线挂高', '方位角',
       '工作频段', '归属区县', '厂家名称', '区域类型', 'distance', '覆盖模型距离', '30米覆盖模型距离',
       '使用覆盖半径', 'distance_use', 'distance_yb', 'h_use', '高度建筑比']

In [ ]:
#- 视野 - 在1400*900的分辨率下  view = 70米对应：125公里（差不多能看到武汉主城区）  700对应1200公里（差不多能看到湖北省）
view = 50000

# 整理数据

In [ ]:
#添加标注列
data_use['description']=''
for inde_1, name_1 in enumerate(de_col):
    data_use['linshi']=de_col[inde_1]+' : '+data_use[de_col[inde_1]].astype('str')+'\n'
    data_use['description'] = data_use['description']+data_use['linshi']
    data_use.drop(columns='linshi')

In [ ]:
data_sectors= data_use.copy()
#添加lod的polygon
data_sectors['lod_dis']= view
data_sectors['lon1'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(45* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat1'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(45* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['lon2'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(225* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat2'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(225* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['list_data']=[([lon_1, lat_1,lon_2, lat_2]) 
 for lon_1, lat_1,lon_2, lat_2 in zip(data_sectors['lon1'],data_sectors['lat1'],data_sectors['lon2'],data_sectors['lat2'])]

In [ ]:
# df = data_sectors.copy()
# #添加颜色列-数字
# df = data_sectors.copy()
# df['colour']=1
# df['colour'] = pd.cut(df['colour'],bins=[-0.1, 0.7, 0.936, 1.1],labels =['red', 'yellow', 'green'])
# df['colour'] = df['colour'].astype('object')
# # df.loc[df[rsrp_count]<100,'colour']='white'

In [ ]:
df = data_sectors.copy()
#添加颜色列-文字
df['colour']='white'
# for name_col in colour_dic:
#     df.loc[df[colour_data]==name_col,'colour'] =colour_dic[name_col]

In [ ]:
#最终使用的列
df_use = df.reindex(columns=[city, 'list_data','colour','description','geometry'])   

# 生成图层

In [ ]:
#完整的多边形
def make_kml(data_t,name='红色栅格',cc='ff0000ff',xiankuan=0,namea='grid_id',list_data = 'list_data',description='description',minlodpixels=1):
    print(data_t.shape,cc,xiankuan,namea,list_data,description,minlodpixels)
    style = simplekml.Style()
    style.linestyle.color = simplekml.Color.changealphaint(220, cc)  # 最终线条上色
    style.polystyle.outline = xiankuan
    style.polystyle.color = simplekml.Color.changealphaint(125, cc )  # 最终形状上色
    lod1 = simplekml.Lod(minlodpixels=minlodpixels, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
    grid_red = kml.newfolder(name=name)
    for grid,list_data ,description_str,geo in zip(data_t[namea],data_t[list_data],data_t[description],data_t['geometry']):
#         pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])#栅格使用
        pol_r = grid_red.newmultigeometry(name=grid)
        pol_r.newpolygon(outerboundaryis=list(geo.exterior.coords),innerboundaryis=[po.coords for po in list(geo.interiors)])
        pol_r.description = description_str
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        lon_dd,lat_dd,lon1_dd,lat1_dd = list_data
        latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                              minaltitude=None, maxaltitude=None, altitudemode=None)
        pol_r.region.latlonaltbox = latlonaltbox
        pol_r.region.lod = lod1
        pol_r.style=style

In [ ]:
#复杂的多边形
def make_kml(data_t,name='红色栅格',cc='ff0000ff',xiankuan=0,namea='grid_id',list_data = 'list_data',description='description',minlodpixels=1):
    print(data_t.shape,cc,xiankuan,namea,list_data,description,minlodpixels)
    style = simplekml.Style()
    style.linestyle.color = simplekml.Color.changealphaint(200, cc)  # 最终线条上色
    style.polystyle.outline = xiankuan
    style.polystyle.color = simplekml.Color.changealphaint(150, cc )  # 最终形状上色
    lod1 = simplekml.Lod(minlodpixels=minlodpixels, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
    grid_red = kml.newfolder(name=name)
    for grid,list_data ,description_str,geo in zip(data_t[namea],data_t[list_data],data_t[description],data_t['geometry']):
#         pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])#栅格使用
        if geo.type == 'MultiPolygon':
            pol_r = grid_red.newmultigeometry(name=grid)
            for geo_t in geo: # Generate longitude values
                pol_r.newpolygon(name=grid,outerboundaryis=list(geo_t.exterior.coords),innerboundaryis=[po.coords for po in list(geo_t.interiors)])
        else:
            pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list(geo.exterior.coords),innerboundaryis=[po.coords for po in list(geo.interiors)])
        pol_r.description = description_str
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        lon_dd,lat_dd,lon1_dd,lat1_dd = list_data
        latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                              minaltitude=None, maxaltitude=None, altitudemode=None)
        pol_r.region.latlonaltbox = latlonaltbox
        pol_r.region.lod = lod1
        pol_r.style=style

In [ ]:
for name_region,data_region in df_use.groupby(city):
    kml = simplekml.Kml()
    file = '{}覆盖区域_{}_{}.kmz'.format(out_file,name_region,data_region.shape[0])
    for name_c,data_t in data_region.groupby('colour'):#red:ff0000ff,green:ff008000,yellow:ff00ffff,white:ffffffff,bule:ffff0000
        if 'red' ==name_c:
            make_kml(data_t,name='红色{}个'.format(data_t.shape[0]),cc='ff0000ff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'blue' == name_c:
            make_kml(data_t,name='蓝色{}个'.format(data_t.shape[0]),cc='ffff0000',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'yellow' == name_c:
            make_kml(data_t,name='黄色{}个'.format(data_t.shape[0]),cc='ff00ffff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'white' == name_c:
            make_kml(data_t,name='白色{}个'.format(data_t.shape[0]),cc='ffffffff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'green'  == name_c:
            make_kml(data_t,name='绿色{}个'.format(data_t.shape[0]),cc='ff008000',namea=use_id,minlodpixels=50,xiankuan=1)
        kml.save(file) if 'kml' in file else kml.savekmz(file, False)
    print('生成图层成功存放在{}'.format(file))

In [ ]:
mytools.sound()
print('自爆')
assert 0 

# 做buffer生成kml

In [ ]:
data = pd.read_clipboard()

In [ ]:
data_buffer = mytools.gisn.add_buffer(data,'lon','lat',100)

# 设置

In [ ]:
#导出目录
otu_file = 'd:/res.kmz'
#- 标注 -添加标注列
de_col = list(data.columns)
data_buffer['description']=''
for inde_1, name_1 in enumerate(de_col):
    data_buffer['linshi']=de_col[inde_1]+' : '+data[de_col[inde_1]].astype('str')+'\n'
    data_buffer['description'] = data_buffer['description']+data_buffer['linshi']
    data_buffer.drop(columns='linshi')
#固定变量
xiankuan = 1

In [ ]:
#最终使用的列
data_use = data_buffer.reindex(columns=['id', 'colour','description','geometry'])   

# 生成图层

In [ ]:
kml = simplekml.Kml()
dic_use = {'红':'ff0000ff','蓝':'ffff0000','黄':'ff00ffff','白':'ffffffff','绿':'ff008000'}
for colourname,data_t in data_buffer.groupby('colour'):
    out_file_use = otu_file+'_'+colourname+'.kml'
    style = simplekml.Style()
    colour_str = dic_use[colourname]
    style.linestyle.color = simplekml.Color.changealphaint(220, colour_str)  # 最终线条上色
    style.polystyle.outline = xiankuan
    style.polystyle.color = simplekml.Color.changealphaint(125, colour_str )  # 最终形状上色
    grid_red = kml.newfolder(name=colourname)
    for id_use,des_use,geo in zip(data_t['id'],data_t['description'],data_t['geometry']):
        pol_r = grid_red.newmultigeometry(name=id_use)
        pol_r.newpolygon(outerboundaryis=list(geo.exterior.coords),innerboundaryis=[po.coords for po in list(geo.interiors)])
        pol_r.description = des_use
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        pol_r.style=style
    print(colourname,'色，制作完成')
#     kml.save(out_file_use) if 'kml' in out_file_use else kml.savekmz(out_file_use, False)
kml.save(otu_file) if 'kml' in otu_file else kml.savekmz(otu_file, False)
print('程序结束')

In [ ]:

style.linestyle.color = simplekml.Color.changealphaint(220, cc)  # 最终线条上色
style.polystyle.outline = xiankuan
style.polystyle.color = simplekml.Color.changealphaint(125, cc )  # 最终形状上色
lod1 = simplekml.Lod(minlodpixels=minlodpixels, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
grid_red = kml.newfolder(name=name)
for grid,list_data ,description_str,geo in zip(data_t[namea],data_t[list_data],data_t[description],data_t['geometry']):
#         pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])#栅格使用
    pol_r = grid_red.newmultigeometry(name=grid)
    pol_r.newpolygon(outerboundaryis=list(geo.exterior.coords),innerboundaryis=[po.coords for po in list(geo.interiors)])
    pol_r.description = description_str
    pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
    lon_dd,lat_dd,lon1_dd,lat1_dd = list_data
    latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                          minaltitude=None, maxaltitude=None, altitudemode=None)
    pol_r.region.latlonaltbox = latlonaltbox
    pol_r.region.lod = lod1
    pol_r.style=style
kml.save(file) if 'kml' in file else kml.savekmz(file, False)

# ----------------------------------天线方向图------------------------------------

# 导入数据和导出目录

In [ ]:
out_file = 'e:/'

In [ ]:
data = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/xiaoqufugaimoxing.data')

In [ ]:
data.columns

# 设置变量

In [ ]:
#- 变量 - 设置经纬度和参与循环的字段
lon,lat,use_id,city = ['RRU经度','RRU纬度','小区中文名','所属地区']

In [ ]:
# #- 颜色 - 按照字符设置颜色
# colour_data = 'city'
# colour_dic = {'大':'red','中':'yellow','小':'blue','微':'green','other':'white'}

In [ ]:
#- 标注 -添加标注列
de_col = ['小区CGI(*)', '小区中文名', '所属地区', 'RRU经度', 'RRU纬度', '天线挂高', '方位角', '工作频段',
       '归属区县', '厂家名称', '区域类型', 'distance', '覆盖模型距离', '30米覆盖模型距离', '使用覆盖半径',
       'distance_use', 'type']

In [ ]:
#- 视野 - 在1400*900的分辨率下  view = 70米对应：125公里（差不多能看到武汉主城区）  700对应1200公里（差不多能看到湖北省）
view = 50000

In [ ]:
data_use = data.copy()

# 整理数据

In [ ]:
#添加标注列
data_use['description']=''
for inde_1, name_1 in enumerate(de_col):
    data_use['linshi']=de_col[inde_1]+' : '+data_use[de_col[inde_1]].astype('str')+'\n'
    data_use['description'] = data_use['description']+data_use['linshi']
    data_use.drop(columns='linshi')

In [ ]:
data_sectors= data_use.copy()
#添加lod的polygon
data_sectors['lod_dis']= view
data_sectors['lon1'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(45* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat1'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(45* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['lon2'] = data_sectors[lon] + data_sectors['lod_dis']*np.sin(225* np.pi/180)*180/( np.pi * 6371229 * np.cos(data_sectors[lat] * np.pi/180))
data_sectors['lat2'] = data_sectors[lat] + data_sectors['lod_dis']*np.cos(225* np.pi/180) / ( np.pi * 6371229 / 180)
data_sectors['list_data']=[([lon_1, lat_1,lon_2, lat_2]) 
 for lon_1, lat_1,lon_2, lat_2 in zip(data_sectors['lon1'],data_sectors['lat1'],data_sectors['lon2'],data_sectors['lat2'])]

In [ ]:
df = data_sectors.copy()
#添加颜色列-文字
df['colour']='white'
# for name_col in colour_dic:
#     df.loc[df[colour_data]==name_col,'colour'] =colour_dic[name_col]

In [ ]:
df.columns

In [ ]:
df

In [ ]:
#最终使用的列
df_use = df.reindex(columns=[ '小区中文名','list_data','colour','description','geometry'])   

# 生成图层

In [ ]:
#完整的多边形
def make_kml(data_t,name='红色栅格',cc='ff0000ff',xiankuan=0,namea='grid_id',list_data = 'list_data',description='description',minlodpixels=1):
    print(data_t.shape,cc,xiankuan,namea,list_data,description,minlodpixels)
    style = simplekml.Style()
    style.linestyle.color = simplekml.Color.changealphaint(220, cc)  # 最终线条上色
    style.polystyle.outline = xiankuan
    style.polystyle.color = simplekml.Color.changealphaint(125, cc )  # 最终形状上色
    lod1 = simplekml.Lod(minlodpixels=minlodpixels, maxlodpixels=-1,minfadeextent=None, maxfadeextent=None)
    grid_red = kml.newfolder(name=name)
    for grid,list_data ,description_str,geo in zip(data_t[namea],data_t[list_data],data_t[description],data_t['geometry']):
#         pol_r = grid_red.newpolygon(name=grid,outerboundaryis=list_data[0])#栅格使用
        pol_r = grid_red.newmultigeometry(name=grid)
        pol_r.newpolygon(outerboundaryis=list(geo.exterior.coords),innerboundaryis=[po.coords for po in list(geo.interiors)])
        pol_r.description = description_str
        pol_r.altitudemode = simplekml.AltitudeMode.clamptoground
        lon_dd,lat_dd,lon1_dd,lat1_dd = list_data
        latlonaltbox = simplekml.LatLonAltBox(east =lon_dd ,north=lat_dd ,south=lat1_dd ,west=lon1_dd,
                                              minaltitude=None, maxaltitude=None, altitudemode=None)
        pol_r.region.latlonaltbox = latlonaltbox
        pol_r.region.lod = lod1
        pol_r.style=style

In [ ]:
for name_region,data_region in df_use.groupby('colour'):
    kml = simplekml.Kml()
    file = '{}覆盖区域_{}_{}.kmz'.format(out_file,name_region,data_region.shape[0])
    for name_c,data_t in data_region.groupby('colour'):#red:ff0000ff,green:ff008000,yellow:ff00ffff,white:ffffffff,bule:ffff0000
        if 'red' ==name_c:
            make_kml(data_t,name='红色{}个'.format(data_t.shape[0]),cc='ff0000ff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'blue' == name_c:
            make_kml(data_t,name='蓝色{}个'.format(data_t.shape[0]),cc='ffff0000',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'yellow' == name_c:
            make_kml(data_t,name='黄色{}个'.format(data_t.shape[0]),cc='ff00ffff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'white' == name_c:
            make_kml(data_t,name='白色{}个'.format(data_t.shape[0]),cc='ffffffff',namea=use_id,minlodpixels=50,xiankuan=1)
        elif 'green'  == name_c:
            make_kml(data_t,name='绿色{}个'.format(data_t.shape[0]),cc='ff008000',namea=use_id,minlodpixels=50,xiankuan=1)
        kml.save(file) if 'kml' in file else kml.savekmz(file, False)
    print('生成图层成功存放在{}'.format(file))